In [ ]:
from model.modelorchastrator import ModelOrchestrator
import pandas as pd
from data_pipeline.dataset import Dataset
from data_pipeline.pipelinesteps import data_splitter
from hydra.utils import instantiate
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from functools import partial
from data_pipeline.pipelinesteps import load_data
from pathlib import Path
from sklearn.ensemble import HistGradientBoostingClassifier
from model.feature_selector import FeatureSelector
from model.modelorchastrator import OptBinningTransformer
import os

In [ ]:

GlobalHydra.instance().clear()
initialize(config_path="../", version_base=None)
cfg = compose(config_name="config")

In [ ]:

base_path = Path(os.getcwd()).parent
dataset = Dataset.create_from_pipeline(
                partial(load_data, base_path), 
                instantiate(cfg.data_pipeline),
                data_splitter=data_splitter,
                target_column="target",
        )

In [ ]:
model_orchestrator = ModelOrchestrator(cfg)

In [ ]:
pipe = model_orchestrator.modelpipeline

In [ ]:
# pipe.fit(dataset.X_train, dataset.y_train)

In [ ]:
from optuna import create_study
from optuna import Trial
from  optuna import samplers

from sklearn.metrics import roc_auc_score


def objective(trial : Trial) -> float:
  model_orchestrator = ModelOrchestrator(cfg, trial)
  pipe = model_orchestrator.create_tuning_pipeline()
  pipe.fit(dataset.X_train, dataset.y_train)
  y_pred = pipe.predict_proba(dataset.X_test)[:,1]
  auc = roc_auc_score(dataset.y_test, y_pred)
  return auc

study = create_study(study_name='optimization', direction='maximize', sampler = samplers.TPESampler())
study.optimize(lambda trial: objective(trial), n_trials=14)

In [ ]:
for i, j in zip(study.best_params.keys(), study.best_params.values()):
    print(f"{i}: {j}")

In [ ]:
study.best_params.keys()